# Split (분할)
![rag_split](figures/rag_split.png)

- Load 한 문서를 지정한 기준의 덩어리(chunk)로 나눈다.
- LLM에 전체 내용을 다보는 것은 비효율 적이다. 그래서 문서를 적당한 size로 나눈 뒤 질문과 연관있는 chunk만 전송하도록 한다.
  - 로딩한 문서에서 질문과 관련되어 참조할 내용들을 찾아서 LLM 모델에 전송하는 것이 목적이다. 그런데 내용이 너무 크면 질문과 관련없는 내용들도 같이 전송되게 된다. 또한 OpenAI와 같은 폐쇄형 LLM에 요청할 경우 비용이 증가의 원인이 된다. 

## 주요 Spliter
- https://api.python.langchain.com/en/latest/text_splitters_api_reference.html

### CharacterTextSplitter
가장  기본적은 Text spliter
- 한 개의 구분자를 기준으로 분리한다.
- chunk size: 분리된 chunk의 최대 글자수를 지정한다.
- overlap 기능이 있어 이전 chunk의 뒷부분이 시작 부분에 겹쳐 나오도록해서 context가 애매하게 끝나지 않게 한다.  
- 주요 파라미터
    - chunk_size: chunk 최대 길이를 지정.
    - chunk_overlap: chunk 간의 겹치는 문자 수를 설정
    - seperator: 구분 문자열을 지정. 기본값: "\n\n"
- 구분자를 기준으로 문서를 나눈다. (나뉜 것을 chunk라고 함) 나뉜 문서의 글자수가 chunk_size 보다 적으면 다음 나뉜 문서와 합친다. (단, 합친 글자수가 chunk_size이하 여야함.)
- 구분자로 나뉜 문서의 글자수가 chunk_size 보다 큰 경우는 chunk_size 는 무시된다.

### RecursiveCharacterTextSplitter
- RecursiveCharacterTextSplitter는 긴 텍스트를 가장 효율적으로 분할한다.
- 여러 구분자를 순차적으로 적용하여 각 chunk가 지정된 최대 길이를 초과하지 않도록 분할한다.
- 분할 기준 문자
    1. 두 개의 줄바꿈 문자 ("\n\n")
    2. 한 개의 줄바꿈 문자 ("\n")
    3. 공백 문자 (" ")
    4. 빈 문자열 ("")
- 방식
    1. '\n\n' (엔터두개) 를 기준으로 분할 한다.
    2. 분할 된 조각(chunk)이 지정한  chunk 크기보다 클 경우 다음 분할 구분 문자인 '\n' 으로 다시 분할 한다.
    3. 그래도 chunk size를 초과하면 공백문자로 분할하고 그래도 크면 다음 기준 문자인 빈문자로 분할 해 지정한 chunk size를 넘지 않도록 한다.       
- 주요 파라미터
    - chunk_size: chunk 최대 길이를 지정.
    - chunk_overlap: chunk 간의 겹치는 문자 수를 설정 이를 통해 문맥이 중간에 끊기지 않도록 한다.
    - separator**s**: 기본 분할 구분 문자 이외의 것들을 추가적으로 지정할 수있다.

In [1]:
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [2]:
path = "data/olympic.txt"
loader = TextLoader(path, encoding="utf-8")
docs = loader.load()
print(type(docs), len(docs), type(docs[0]))
print(docs[0].page_content[:200])
print(docs[0].metadata)

<class 'list'> 1 <class 'langchain_core.documents.base.Document'>
올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열
{'source': 'data/olympic.txt'}


In [3]:
# 1. splitter 생성
splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50, # 겹치는 글자수 (default: 0)
    # length_functon=len, # chunk size 를 계산할 떄 사용할 함수 : default: len()
    # separator="", # chunk를 나눌때 사용할 구분 문자열(default:\n\n)
)

# 2. chunk로 나누기
## split_text(str) -> list[str] : string 문자열로 넣어서 나눔.
## split_documents(list[Document]) -> list[Document] : Document를 받아서 나눔.
split_docs = splitter.split_documents(docs)

splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)

print(len(docs), len(split_docs))
print(split_docs[0].page_content)

Created a chunk of size 1726, which is longer than the specified 200
Created a chunk of size 1556, which is longer than the specified 200
Created a chunk of size 1391, which is longer than the specified 200
Created a chunk of size 990, which is longer than the specified 200
Created a chunk of size 413, which is longer than the specified 200
Created a chunk of size 530, which is longer than the specified 200
Created a chunk of size 590, which is longer than the specified 200
Created a chunk of size 793, which is longer than the specified 200
Created a chunk of size 1159, which is longer than the specified 200
Created a chunk of size 428, which is longer than the specified 200
Created a chunk of size 362, which is longer than the specified 200
Created a chunk of size 1376, which is longer than the specified 200
Created a chunk of size 674, which is longer than the specified 200
Created a chunk of size 1401, which is longer than the specified 200
Created a chunk of size 1105, which is lon

1 18
올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.
또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 데플림픽, 10대 선수들이 참여하는 유스 올림픽 등을 들 수 있다. 그 뿐만 아니라 IOC는 20세기의 변화하는 

In [4]:
# RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    # separators=["\n\n", "\n", " ", "|", ...], 
    ## 구분자들을 순서대로 지정. 중국어/일본어 처럼 공백을 구분자로 쓰지 않는 언어들은 그에 맞는 구분자를 지정해준다.
)

split_docs = splitter.split_documents(docs)
print(len(split_docs))
len(split_docs[0].page_content)
print(split_docs[0].page_content)

28
올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.


## Token 수 기준으로 나누기

- LLM 언어 모델들은 입력 토큰 수 제한이 있어서 요청시 제한 토큰수 이상의 프롬프트는 전송할 수 없다.
- 따라서 텍스트를 chunk로 분할할 때는 글자수 보다 **토큰 수를 기준으로 크기를 지정하는 것**이 좋다.  
- 토큰기반 분할은 텍스트의 의미를 유지하면서 분할하는 방식이므로 문자 기반 분할과 같이 단어가 중간잘리는 것들을 방지할 수 있다. 
- 토큰 수 계산할 때는 사용하는 언어 모델에 사용된 것과 동일한 tokenizer를 사용하는 것이 좋다.
  - 예를 들어 OpenAI의 GPT 모델을 사용할 경우 tiktoken 라이브러리를 활용하여 토큰 수를 정확하게 계산할 수 있다.

### [tiktoken](https://github.com/openai/tiktoken) tokenizer 기반 분할
- OpenAI에서 GPT 모델을 학습할 때 사용한 `BPE` 방식의 tokenizer. **OpenAI 언어모델을 사용할 경우 이것을 사용하는 것이 좀 더 정확하게 토큰계산할 수있다.**
- Splitter.from_tiktoken_encoder() 메소드를 이용해 생성
  - `RecursiveCharacterTextSplitter.from_tiktoken_encoder()`
  - `CharacterTextSplitter.from_tiktoken_encoder()`
- 파라미터
  - encode_name: 인코딩 방식(토큰화 규칙)을 지정. OpenAI는 GPT 모델들 마다 다른 방식을 사용했다. 그래서 사용하려는 모델에 맞는 인코딩 방식을 지정해야 한다.
    - `cl100k_base`: GPT-4 및 GPT-3.5-Turbo 모델에서 사용된 방식.
    - `r50k_base:` GPT-3 모델에서 사용된 방식 
  - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- tiktoken 설치
  - `pip install tiktoken`

In [ ]:
# !pip show tiktoken

Name: tiktoken
Version: 0.8.0
Summary: tiktoken is a fast BPE tokeniser for use with OpenAI's models
Home-page: https://github.com/openai/tiktoken
Author: Shantanu Jain
Author-email: shantanu@openai.com
License: MIT License

Copyright (c) 2022 OpenAI, Shantanu Jain

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULA

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",  # OpenAI 모델 학습시 사용한 titoken 토크나이가 다름. 그래서 gpt 모델을 지정함.
    chunk_size=1000,   # token 수(글자수가 아님) -> 토큰계산은 tiktoken 토크나이저를 이용.
    chunk_overlap=100,
)
split_docs2= splitter.split_documents(docs)
print(len(split_docs), len(split_docs2))

28 18


In [7]:
print(len(split_docs2[0].page_content)) # 토큰수 기준
print(len(split_docs[0].page_content))  # 글자수 갯수

1463
808


### HuggingFace Tokenizer
- HuggingFace 모델을 사용할 경우 그 모델이 사용한 tokenizer를 이용해 토큰 기반으로 분할 할 수있다.
- `from_huggingface_tokenizer()` 메소드를 이용해 생성
- 파라미터
  - tokenizer: HuggingFace tokenizer 객체
  - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- `transformers` 라이브러리를 설치해야 한다.
  - `pip install transformers` 

In [ ]:
# !pip show transformers

In [8]:
from transformers import AutoTokenizer
model_id = "beomi/kcbert-base" # Huggingace 모델로 사용할 llm모델 tokenizer를 지정함.

# 토크나이저를  HuggingFace 모델 허브에서 받아 생성함.
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer))


<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [20]:
splitter_hf = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=1000,
    chunk_overlap=100
)
split_docs3 = splitter_hf.split_documents(docs)
print(len(split_docs3), len(split_docs2), len(split_docs))
print(len(split_docs3[0].page_content))
print(split_docs3[0].page_content)

12 18 28
1726
올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.
또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 데플림픽, 10대 선수들이 참여하는 유스 올림픽 등을 들 수 있다. 그 뿐만 아니라 IOC는 20